In [35]:
import os
import sys
import numpy as np
import pandas as pd
import importlib
from shutil import rmtree, copy, copytree
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib import rcParams
from matplotlib.ticker import ScalarFormatter

rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['DejaVu Serif']
rcParams['mathtext.fontset'] = 'dejavuserif'
rcParams['font.size'] = 30
rcParams['axes.formatter.use_mathtext'] = True
rcParams['axes.unicode_minus'] = True
rcParams['mathtext.default'] = 'regular'
rcParams['text.usetex'] = False

from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy.visualization import (PercentileInterval, LogStretch, PowerStretch, ManualInterval, ZScaleInterval, ImageNormalize)
from astroquery.vizier import Vizier
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u

from casatasks import listobs, split, clearcal, delmod, rmtables, setjy, gaincal, bandpass, applycal, tclean, flagdata, ft, fixvis, phaseshift, casalog, mstransform, exportfits
from casatools import componentlist, msmetadata, imager, ms, table

In [36]:
# Set path for measurement set, analysis output, and CASA log files
basepath = '/data/jfaber/dsa110-contimg/sandbox/2025-03-18/'
casalog.setlogfile(f'{basepath}/casa_logfile.log')

True

In [39]:
# Identify the base measurement sets and sorted them in time
msfiles = [file for file in os.listdir(basepath) if file.endswith('_base.ms')]
msfiles_sorted = sorted(msfiles, key=lambda fname: float(fname.split("_ra")[1].split("_")[0]))
print(msfiles_sorted)
msfile = msfiles_sorted[3]

msfile_bcal = msfile.split(".ms")[0][:-5]+f'_bcal_sb{chanstoavg}.ms'
print(f'MS File to rephase to bandpass calibrator: {msfile_bcal}')
print('\n')

#if os.path.exists(os.path.join(basepath, msfile_bcal)):
#    rmtree(os.path.join(basepath, msfile_bcal))
#    #copytree(os.path.join(basepath, msfile_image), os.path.join(basepath, msfile_bcal))
#split(vis=os.path.join(basepath, msfile), outputvis=os.path.join(basepath, msfile_bcal), datacolumn='all', width=chanstoavg)

#for msfile in msfiles:
print('\n')
print(f'Base MS File: {msfile}')
print('\n')

['2025-03-18T01:30:21_ra080.4_dec+16.5_base.ms', '2025-03-18T06:52:15_ra161.2_dec+16.6_base.ms', '2025-03-18T06:57:25_ra162.4_dec+16.6_base.ms', '2025-03-18T07:02:34_ra163.7_dec+16.6_base.ms', '2025-03-18T07:07:43_ra165.0_dec+16.6_base.ms', '2025-03-18T18:54:55_ra342.4_dec+16.4_base.ms', '2025-03-18T19:00:04_ra343.6_dec+16.4_base.ms', '2025-03-18T19:05:13_ra344.9_dec+16.4_base.ms']
MS File to rephase to bandpass calibrator: 2025-03-18T07:02:34_ra163.7_dec+16.6_bcal_sb8.ms




Base MS File: 2025-03-18T07:02:34_ra163.7_dec+16.6_base.ms




In [40]:
msdate = msfile.split('_')[0]
chanstoavg = 8 #set to 4 later
field_image_idx = 11
field_cal_idx0, field_cal_idx1 = 0, 23
uvrange = '0.3klambda'
refant = 'pad103'

print('\n')
print(f'Observation Date: {msdate}')
print(f'No. Channels to Average: {chanstoavg}')
print(f'Central Field: Field {field_image_idx}')
print(f'Fields to Calibrate Over: Field {field_cal_idx0} to Field {field_cal_idx1}')
print('\n')



Observation Date: 2025-03-18T07:02:34
No. Channels to Average: 8
Central Field: Field 11
Fields to Calibrate Over: Field 0 to Field 23




In [41]:
msmd = msmetadata()
msmd.open(os.path.join(basepath, msfile))
fieldnames = msmd.fieldnames()

fieldname_image = fieldnames[field_image_idx]
fieldname_cal_list = fieldnames[field_cal_idx0: field_cal_idx1]
fieldname_cal = ', '.join(fieldname_cal_list)

In [28]:
sys.path.insert(0, '/data/jfaber/dsa110-contimg/sandbox/scripts')
import make_skymodel_cl_mf
importlib.reload(make_skymodel_cl_mf)
from make_skymodel_cl_mf import make_skymodel

# Number of bright sources to include in the sky model
top_n = 200

print(f'Generating a sky model with the top {top_n} brightest NVSS sources...')

# Define filename strings
ra_str = f"{msfile.split('_')[1][2:5]}p{msfile.split('_')[1][6:9]}"
dec_str = f"{msfile.split('_')[2][4:6]}p{msfile.split('_')[2][7:8]}"
clfile = f'nvss_top{top_n}_{ra_str}_{dec_str}.cl'
cllabel = clfile.split('.')[0]

if os.path.exists(os.path.join(basepath, f'nvss_top{top_n}_{ra_str}_{dec_str}.cl')):
    rmtree(os.path.join(basepath, f'nvss_top{top_n}_{ra_str}_{dec_str}.cl'))

nvss_catalog = make_skymodel(basepath, msfile_bcal, fieldname=fieldname_image, top_n=top_n, pbfrac=0.5)

Generating a sky model with the top 200 brightest NVSS sources...

Primary Beam FWHM (2.946 degrees):
  RA:  78.936186 to 81.881836
  Dec: 15.005736 to 17.951386
Querying VIII/65/nvss/ ...
Found 446 sources in the field!
Querying J/other/A+A/598/A78/table3 ...
No sources found in the TGSS ADR1 catalog.
Creating new component list 'nvss_top200_080p4_16p5.cl'...
Component list 'nvss_top200_080p4_16p5.cl' created successfully with 200 sources.


2025-03-19 23:08:13	WARN	componentlist::close	componentlist closed


In [29]:
sys.path.insert(0, '/data/jfaber/dsa110-contimg/sandbox/scripts')
import image_skymodel_cl
importlib.reload(image_skymodel_cl)
from image_skymodel_cl import image_skymodel

print('Imaging the sky model and saving plot...')

wcs_2d_coords_x, wcs_2d_coords_y, phasecenter = image_skymodel(basepath, msfile_bcal, fieldname_image, nvss_catalog, top_n=top_n, nx=4800, ny=4800, cellx='3arcsec', celly='3arcsec', mode='mfs', phasecenter=None)
startmodel_nvss = cllabel + '.image'
print(f'Phase center of the total field: {phasecenter}')
print('\n')

Imaging the sky model and saving plot...
Image Parameters
----------------
nx, ny: (4800, 4800)
cellx, celly: ('3arcsec', '3arcsec')
mode: mfs
Phase Center: J2000 5h21m44.621490721007504s 16d28m43.055491020262764s
MS Selected: /data/jfaber/dsa110-contimg/sandbox/2025-03-18/2025-03-18T01:30:21_ra080.4_dec+16.5_bcal_sb8.ms


2025-03-19 23:08:38	WARN	imager::setdata()	Ignoring that ms/data/jfaber/dsa110-contimg/sandbox/2025-03-18/2025-03-18T01:30:21_ra080.4_dec+16.5_bcal_sb8.msspecified here
2025-03-19 23:08:38	WARN	imager::setdata()	Imager was constructed with an ms 
2025-03-19 23:08:38	WARN	imager::setdata()	if multi-ms are to be used please construct imager without parameters and use setdata to specify the ms's and selection


Phase center of the total field: J2000 5h21m44.621490721007504s 16d28m43.055491020262764s




Set OBSGEO-B to    37.280380 from OBSGEO-[XYZ].
Set OBSGEO-H to     2193.256 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [30]:
print('Applying bandpass solutions to non-centered MS...')
print('\n')

bcalfield = fieldname_cal
bcalfile = 'J2253+1608_2025-03-18T19:00:04_f0f23.bcal'

print(f'Not calculating bandpass solutions! Using previously calculated solution table {bcalfile}...')
print('\n')

applycal(vis=os.path.join(basepath, msfile_bcal),
        field=fieldname_cal,
        gaintable=os.path.join(basepath, bcalfile))

Applying bandpass solutions to non-centered MS...


Not calculating bandpass solutions! Using previously calculated solution table J2253+1608_2025-03-18T19:00:04_f0f23.bcal...




In [31]:
print('Copying non-centered bandpass-corrected MS for next steps (only corrected column)...')
msfile_bcal_corr = msfile_bcal.split(".ms")[0] + '_corr.ms'
split(vis=os.path.join(basepath, msfile_bcal), outputvis=os.path.join(basepath, msfile_bcal_corr), datacolumn='corrected', width=chanstoavg)

Copying non-centered bandpass-corrected MS for next steps (only corrected column)...


In [32]:
print('Placing the sky model in the MODEL column of the non-centered MS...')
print('\n')

ft(vis=os.path.join(basepath, msfile_bcal_corr), 
    field=fieldname_cal,
    complist=os.path.join(basepath, clfile), 
    reffreq='1.4GHz',
    usescratch=True)

Placing the sky model in the MODEL column of the non-centered MS...




In [33]:
print(f'Copy the MS and place the phasecenter for all columns on the center of the total field at {phasecenter}...')
print('\n')

msfile_bcal_cntrd = msfile_bcal_corr.split(".ms")[0]+'_cntrd.ms'

#channelstoavg=16
if os.path.exists(os.path.join(basepath, msfile_bcal_cntrd)):
    rmtree(os.path.join(basepath, msfile_bcal_cntrd))
    rmtree(os.path.join(basepath, msfile_bcal_cntrd.split(".ms")[0]+'.flagversions'))

#mstransform(vis=os.path.join(basepath, msfile_bcal), outputvis=os.path.join(basepath, msfile_bcal_cntrd), phasecenter=calibrator_coord_hms)
mstransform(vis=os.path.join(basepath, msfile_bcal_corr), outputvis=os.path.join(basepath, msfile_bcal_cntrd), phasecenter=phasecenter, datacolumn='all')

Copy the MS and place the phasecenter for all columns on the center of the total field at J2000 5h21m44.621490721007504s 16d28m43.055491020262764s...




In [ ]:
print('Performing phase-only gain calibration on non-centered MS...')
print('\n')

#bcalfield = fieldname
pcalfield = fieldname_cal

#if pcalfield is None:
#    pcalfile = f'{calibrator_name}_{msdate}_allfields.pcal'
#else:
#    pcalfile = f'{calibrator_name}_{msdate}_f{field_cal_idx0}f{field_cal_idx1}.pcal'
#pcalfile = 'J1053+1627_2025-03-18T07:02:34_f0f23.pcal'

if os.path.exists(os.path.join(basepath, pcalfile)):
    rmtree(os.path.join(basepath, pcalfile))

if pcalfield is None:

    gaincal(vis=os.path.join(basepath, msfile_bcal_cntrd),
            field='',
            caltable=os.path.join(basepath, pcalfile),
            refant=refant,
            solint='inf',
            calmode='p',
            gaintype='G',
            minsnr=3,
            #gaintable = ['J1132+1628_2025-03-02T08:43:06_f0f23.bcal'],
            combine='scan, obs, field',
            uvrange='>'+uvrange)

else:

    gaincal(vis=os.path.join(basepath, msfile_bcal_cntrd),
            field=pcalfield,
            caltable=os.path.join(basepath, pcalfile),
            refant=refant,
            solint='inf',
            calmode='p',
            gaintype='G',
            minsnr=3,
            #gaintable = ['J1132+1628_2025-03-02T08:43:06_f0f23.bcal'],
            combine='scan, obs, field',
            uvrange='>'+uvrange)

Performing phase-only gain calibration on non-centered MS...




In [42]:
pcal = 'J1053+1627_2025-03-18T07:02:34_f0f23.pcal'
msfile_bcal_cntrd = '2025-03-18T07:02:34_ra163.7_dec+16.6_bcal_sb8_corr_cntrd.ms'

applycal(vis=os.path.join(basepath, msfile_bcal_cntrd),
        field=fieldname_cal,
        gaintable=os.path.join(basepath, pcalfile))